<a href="https://colab.research.google.com/github/kaodamie/PAN-CLEF---Profilling-Hate-Spreaders-on-Twiter/blob/main/PAN21_Exp__Spanish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q tensorflow-text

In [ ]:
pip install -q tf-models-official

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os 
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as mpl
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow import feature_column
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import shutil
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization 
tf.get_logger().setLevel('ERROR')

name=r'/content/drive/MyDrive/Dataset/pan21-author-profiling-training-2021-03-14/es//'

xml_text=[]
list_filenames=[]
for file in os.listdir(name):
        filename = os.fsdecode(file)
        
        if filename.endswith('.xml'):
#             print(filename)   
            list_filenames.append(filename.split(sep='.')[0])
            try:
                tree = ET.parse(name+file) 
                root =tree.getroot() 
                xml_document=[]
                for neighbor in root.iter('document'):
                    xml_document.append(neighbor.text)
                     
                    #print(xml_document)
#                 print('\n\n')
                xml_text.append(" ".join(xml_document))
            except Exception as e:
                print(e, name+file)
#                 display(e, folder+file)
        
        elif filename.endswith('txt'):
            fileObject = open(name+file)
            data = fileObject.read().splitlines()
            dat=[]
            for i in data:
                dat.append(i.split(sep=':::'))
                

In [ ]:
df1=pd.DataFrame(zip(list_filenames,xml_text),columns=('ids','xml_text')).set_index('ids').join(pd.DataFrame(dat,columns=('ids','labels')).set_index(['ids']))
# v=pd.DataFrame(dat)
# v['xml'] =xml_text


# in future make a k-fold cross validation with sklearn
train, test = train_test_split(df1, test_size=0.2)
val, test = train_test_split(test, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

train_ds=train['xml_text'].astype(str).to_numpy()
val_ds=val['xml_text'].astype(str).to_numpy()

whole_dataset=df1['xml_text'].astype(str).to_numpy()
whole_labels=df1['labels'].astype(int).to_numpy()

train_labels=train['labels'].astype(int).to_numpy()
val_labels=val['labels'].astype(int).to_numpy()

In [ ]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((train_ds,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((val_ds,val_labels))

whole_ds=tf.data.Dataset.from_tensor_slices((whole_dataset,whole_labels))

train_ds = train_ds.batch(BATCH_SIZE)
valid_ds = valid_ds.batch(BATCH_SIZE)

# PREFETCH

train_ds = train_ds.shuffle(5).prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.shuffle(5).prefetch(buffer_size=AUTOTUNE)
whole_ds=whole_ds.shuffle(5).prefetch(buffer_size=AUTOTUNE)

In [ ]:
#google sentence encoder classifier
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2",name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input) 
  encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2",name='encoder')

  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)

  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()

In [ ]:
classifier_model.compile(optimizer='adam',
                         loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                         metrics=tf.metrics.BinaryAccuracy())

mcp_save = ModelCheckpoint('multi_sentence_encoder.h5', save_best_only=True, monitor='val_loss', mode='min')
early_stop=EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [ ]:
#print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(train_ds,
                               validation_data=valid_ds,
                               epochs=10,callbacks=[mcp_save, early_stop], verbose=1)



In [ ]:
# classifier_model.save('/content/drive/MyDrive/Dataset/PanResults/es')

In [ ]:
reloaded=tf.saved_model.load('/content/drive/MyDrive/Dataset/PanResults/es_savedmodel')

In [ ]:
reloaded

In [ ]:
train_loss, train_acc = classifier_model.evaluate(train_ds, verbose=1)
validation_loss, validation_acc =classifier_model.evaluate(valid_ds, verbose=1)

In [ ]:
print('* Accuracy on training set: %0.2f%%' % (100 * train_acc))
print('* Accuracy on validation set: %0.2f%%' % (100 * validation_acc))

In [ ]:

results=tf.sigmoid(classifier_model(tf.constant(test['xml_text'])))


results=tf.sigmoid(reloaded(tf.constant(test['xml_text'])))
#test_results=round()

In [ ]:
test['results']=results.numpy()

In [ ]:
#  a sample testset after shuffling
test 


In [ ]:
# another sample test after shuffling
test

In [ ]:
#working with the actual test data
name=r'/content/drive/MyDrive/Dataset/pan21-author-profiling-test-without-gold/es//'

xml_text=[]
list_filenames=[]
for file in os.listdir(name):
        filename = os.fsdecode(file)
        
        if filename.endswith('.xml'):
#             print(filename)   
            list_filenames.append(filename.split(sep='.')[0])
            try:
                tree = ET.parse(name+file) 
                root =tree.getroot() 
                xml_document=[]
                for neighbor in root.iter('document'):
                    xml_document.append(neighbor.text)
                     
                    #print(xml_document)
#                 print('\n\n')
                xml_text.append(" ".join(xml_document))
            except Exception as e:
                print(e, name+file)
#                 display(e, folder+file)

In [ ]:
df2=pd.DataFrame(zip(list_filenames,xml_text),columns=('ids','xml_text')).set_index('ids')

In [ ]:
df2

In [ ]:
# if reusing saved model, 
test_results=tf.sigmoid(reloaded(tf.constant(df2['xml_text'])))

# if not reusing saved model then comment out top
test_results=tf.sigmoid(classifier_model(tf.constant(df2['xml_text'])))

In [ ]:
for i in range(len(df2['xml_text'])):
  print(f'{df2.index[i]}: {round(test_results[i][0].numpy())}')
  root = ET.Element('author', id=df2.index[i],lang='es',type=f'{round(test_results[i][0].numpy())}')
  s = ET.tostring(root)
  print(str(s)) 
  tree = ET.ElementTree(root)
  # change directory
  tree.write("/content/drive/MyDrive/Dataset/PanResults/Test_Results/es/%s.xml" %df2.index[i])